## VRP SHEDULING

In [1]:
from __future__ import print_function
from ortools.linear_solver import pywraplp
import array
import numpy as np
import scipy as sp
from numpy import matrix
import random

## Input data

In [168]:
## TEST DATA - MINI USE CASE

# load dispatsh matrix - demand
    #LJ,MB,CE,KP
D = [11, 60, 22, 70]
C = [100, 100,100]
V = np.size(C)
#network graph, stopci = povezave 12, 23, 13, vrstice = mesta
    #LJ,MB,CE,KP
E = [
    [1, 1, 1, 0, 0],
    [1, 0, 0, 1, 0],
    [0, 1, 0, 1, 1],
    [0, 0, 1, 0, 1]
    ]

edges = len(E[1])
nodes = len (E)
#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]


In [111]:
D = [1, 2, 4]
C = [10]
V = np.size(C)
#network graph, stopci = povezave 12, 23, 13, vrstice = mesta
    #LJ,MB,CE,KP
E = [
    [1, 0, 1],
    [1, 1, 0],
    [0, 1, 1],]
edges = len(E[1])
nodes = len (E)

In [26]:
D = [10, 20, 30]
C = [10,10]
V = np.size(C)
#network graph, stopci = povezave 12, 23, 13, vrstice = mesta
    #LJ,MB,CE,KP
E = [
    [11, 12],
    [21, 22],
    [31, 32],]
edges = len(E[1])
nodes = len (E)

In [213]:
## TEST DATA - SMALL USE CASE
# load dispatsh matrix - demand  #LJ,MB,CE,KP
D = [11, 22, 33, 44, 55, 66]
# vehicle load capacity
C = [80, 80, 80]
V = np.size(C)
#network graph, stopci = povezave 12, 23, 13, vrstice = mesta
    #LJ,MB,CE,KP
E = [
    [1,0,0,0,0,1,0,1,0],
    [1,1,0,0,0,0,1,0,1],
    [0,1,1,0,0,0,0,0,0],
    [0,0,1,1,0,0,0,0,0],
    [0,0,0,1,1,0,0,1,1],
    [0,0,0,0,1,1,0,0,0]]

edges = len(E[1])
nodes = len (E)
if len(D) != nodes:
    print("Error - wrong dimension of dispatch matrix!!")

In [95]:
# test matrix
D = [11, 22, 33, 44]
# vehicle load capacity
C = [100, 100, 100]
V = np.size(C)
E = [
    [11, 12, 13, 14, 15],
    [21, 22, 23, 24, 25],
    [31, 32, 33, 34, 35],
    [41, 42, 43, 44, 45]
    ]
edges = len(E[1])
nodes = len (E)


In [93]:

# load dispatsh matrix - demand
    #LJ, CE, MB, KP, NM
D = [17, 6, 25, 37, 12]
# vehicle load capacity
C = [100, 100, 100]

#network graph, stopci = povezave 12, 23, 34, 45, 35, 13, vrstice = mesta
    #LJ,CE,MB,KP,NM
E = [
    [10, 0, 0, 0, 16, 11],
    [0, 21, 23, 0, 0, 26],
    [31, 32, 0, 0, 0, 0],
    [0, 0, 0, 44, 45, 0],
    [0, 0, 53, 54, 0, 0]]

#E transpose
Et= [[E[j][i] for j in range(len(E))] for i in range(len(E[0]))]
#-E (negative values)
negE=np.negative(E)

#number of nodes
N = np.size(E,0)
#number of edges
M = np.size(E,1)
#number of vehicles
V = np.size(C)

#[0, 0, 1, 1, 0, 0]
E

[[10, 0, 0, 0, 16, 11],
 [0, 21, 23, 0, 0, 26],
 [31, 32, 0, 0, 0, 0],
 [0, 0, 0, 44, 45, 0],
 [0, 0, 53, 54, 0, 0]]

## Creating Constraint matrix A

In [214]:
# CONSTRAINT IV - UPDATED - there is od number of edges on cycles
# the right side of the equation should be zeros(n_edges*n_cycles + n_vert*n_cycles)
# b variables = size([C])= n_cycles*n_edges + n_cycles*n_nodes

size = 2*nodes*V + nodes*V + edges*V
size1 = nodes * V + edges *V 
A1 = np.zeros((size, size1))
b1 = []
for i in range (0, 3*nodes*V+edges*V):
    b1.append(0)
for k in range (0, V):
    for i in range (0, nodes):
        for j in range (0, edges):
            #A1[i*V+k,j]=E[2*i*edges+k*edges+j]
            A1[k*nodes+i,j+k*edges] = E[i][j]
            A1[nodes*V+k*nodes+i,j+k*edges] = -E[i][j]
            A1[k*nodes+i,edges*V+k*nodes+i] = -2
            A1[nodes*V+k*nodes+i,edges*V+k*nodes+i] = 2
            A1[2*nodes*V+k*nodes+i,edges*V+k*nodes+i] = -1
            #A1[3*nodes*V+k*nodes+i,edges*V+k*nodes+i] = -1
for k in range (0, V):  
    for j in range (0, edges):
        A1[3*nodes*V+k*edges+j,k*edges+j] = -1
#print(str('\n'.join(' '.join([str(int(val)) for val in row]) for row in A1)))

In [215]:
# CONSTRAINT II - A2 - the number of packets delivered on the node is equal to all total demand on the node
#number of rows: 2 * num. of nodes
#number of columns: num. nodes * num. of cycles
# variables Oki
# B vector = [D, -D]
b2 = D + [-val for val in D]
for i in range (0, V*nodes):
    b2.append(0)
sizeA2= 2*nodes + nodes*V
sizeA21= nodes*V
A2 = np.zeros((sizeA2, sizeA21))
for i in range (0, nodes):
    for k in range (0, V):
            A2[i,i+nodes*k]=1
            A2[i+nodes,i+nodes*k]=-1
            A2[i*V+2*nodes+k,i*V+k]=-1
            
#CONSTRAINT III A3 matrix: sum of load on each vehicles must not exceed vehicle load capacity
# variables Oki
sizeA3=V
sizeA31=V*nodes
#constants
b3=C.copy()
b23=b2+b3

A3 = np.zeros((sizeA3, sizeA31))
for k in range (0, V):                                  # for each vehicle
    for i in range (0, nodes):                         #take the sum of load on enach node (sum on Ow variables)
        #print("j", j+i*V)       
        A3[k,i+k*nodes]=1
# A23 - concatenating A2 and A3
A23=A2
for i in range (0, len(A3)):
    A23=np.vstack([A23, A3[i,:]])
#print(str('\n'.join(' '.join([str(int(val)) for val in row]) for row in A23)))

In [216]:
#CONSTRAINT I - total number of all packets delivered is equal to summ of all D

size1= 2*nodes*V + edges*V   # number of columns in matrix A1+A2+A3, variables X, K, O
size2= V*edges*nodes         # number of additional columns in matrix A41, variables Aijk
# b vector = d (sum of all parcels). size = 1; D_all = np.sum(D)

# constraint 4.1. SUM for ijk -> (Eij * Aijk)
A41 = np.zeros((1, size1+size2))
l=0
for i in range (0, nodes):              # itteration by node in E 
    for j in range (0, edges):         # itteration by edges in E
        for k in range (0, V):
            A41[0,(size1+l)]=-(E[i][j])
            #print(size1+l)
            l=l+1

In [217]:
# Adding constraints 4.2.: Aijk - Cki*di <= 0 
size1= V * edges * nodes   
size2= 2*V*nodes + V*edges + V*nodes*edges
#b vector = [0,0,...0], size = nodes* edges * V
offsetA4=0
A42 = np.zeros((size1, size2))
for i in range (0, nodes):
    for j in range (0,edges): 
        for k in range (0, V):
            A42[i*edges*V+j*V+k, edges*k+j]=-D[i]
            #print(i*edges*V+j*V+k, edges*k+j)
            A42[i*edges*V+j*V+k, 2*V*nodes + V*edges + i*edges*V+j*V+k]=1

In [218]:
#constraint 4.3.: Aijk - Oki <= 0
cycles=V
size1= cycles * edges + 2*nodes*cycles + cycles*edges*nodes   
size= V*nodes*edges
A43 = np.zeros((size, size1))
#b vector = [0,0,...0], size = 

#offset= V*nodes+V*edges
#offset1= 2*V*nodes+V*edges
offset=0
offset1=edges*V+nodes*V
offset2=edges*V+2*nodes*V
for i in range (0, nodes):
    for j in range (0, edges):
        for k in range (0, V):       
            #print (offset+i*edges*V+j*V+k, offset1 + nodes*k+i)
            A43[offset+i*edges*V+j*V+k, offset1 + nodes*k+i]=-1
            A43[offset+i*edges*V+j*V+k,offset2+i*edges*V+j*V+k]=1

In [219]:
#concatenate A4 matrix
A4=A41.copy()
for i in range (0, len(A42)):
    A4=np.vstack([A4, A42[i,:]])
for i in range (0, len(A43)):
    A4=np.vstack([A4, A43[i,:]])

b4=[-2*np.sum(D)]
for i in range (0, (2*V*nodes*edges)):
    b4.append(0)

In [220]:
#print(str('\n'.join(' '.join([str(int(val)) for val in row]) for row in A4)))
#len(b4)

In [221]:
# FINAL MATRIX  - A with all constraints
#concatenate A1 and A23 = A matrix
A1extend = np.c_[A1, np.zeros((len(A1), len(A23[0])))]
A23extend = np.c_[np.zeros((len(A23), len(A1[0]))), A23]

A123=A1extend.copy()
for i in range (0, len(A23)):
    A123=np.vstack([A123, A23extend[i,:]])

#concatenate A4 matrix
A4=A41.copy()
for i in range (0, len(A42)):
    A4=np.vstack([A4, A42[i,:]])
for i in range (0, len(A43)):
    A4=np.vstack([A4, A43[i,:]])

#Final concate A123 & A4
A123extend = np.c_[A123, np.zeros((len(A123), nodes*edges*V))]
A=A123extend.copy()
for i in range (0, len(A4)):
    A=np.vstack([A, A4[i,:]])
b=b1+b23+b4
non_zero_rows = np.count_nonzero((A != 0).sum(1)); zero_rows = len(A) - non_zero_rows
print ("number of zero rows in A matirx =", zero_rows)
print ("number of variables in A =", len(A[1]))
print ("number of constraints in A =", len(A))


number of zero rows in A matirx = 0
number of variables in A = 225
number of constraints in A = 439


In [222]:
len(b)

439

## Creating & declaring variables

In [223]:
# CREATE VARIABLES  X - vector with c11-cnn variables
# vsak vektor v matriki X1n = dolžine len(E). 
#Skupno število vseh vektorjev X1n = len (Et)
#X variables X[0] to X[len(E) * V -1] -  variables in objective function 
#K - X[len(E) * V] to X[len(E) * V + len(A1) -1]  slack variables
#Ow - cycles load dispatch variables - X[len(E) * V + len(Et)*V*2] to X[len(E) * V + len(Et)*V*3 -1 ] 

# C variables
X1 = []
for i in range (0, edges*V):
    var='x'+str(i)
    X1.append(var)

#K variables
K = []
for j in range (0, V*nodes):
    #creating vector with K variables
    var='k'+str(j)
    K.append(var)
    
#"Ow" variables
Ow = []
for j in range(0, V*nodes):   # numer of vehicles 
    var='Ow'+str(j)
    Ow.append(var)

#"Aijk" variables
Aijk = []
for i in range (0,nodes):
    for j in range(0, edges):   # numer of vehicles 
        for k in range (0, V):
            var='A'+str(i)+str(j)+str(k)
            Aijk.append(var)
    
#Final X vector with all variables
X=X1.copy()
for i in range (0,len(K)):
    X.append(K[i])
for j in range (0,len(Ow)):
    X.append(Ow[j])
for ijk in range (0,len(Aijk)):
    X.append(Aijk[ijk])
print("number of all variables =", len(X))
print("number of C variables =", len(X1))
print("number of K variables =", len(K))
print("number of O variables =", len(Ow))
print("number of Aijk variables =", len(Aijk))

number of all variables = 225
number of C variables = 27
number of K variables = 18
number of O variables = 18
number of Aijk variables = 162


In [224]:
#Declaring the solver
solver = pywraplp.Solver('SolveIntegerProblem',
                           pywraplp.Solver.CBC_MIXED_INTEGER_PROGRAMMING)
x_min = -solver.infinity()                   # lower variables border 
x_max = solver.infinity()                  # Upper variables border                        
    
variables = [] 

for varN, xi_name in enumerate(X1):                          # declaring objective variables [C1.... Cn]
    variables.append(solver.IntVar(0.0, x_max, xi_name))
    
for varN, xi_name in enumerate(K):                          # declaring slack variables
    variables.append(solver.IntVar(0.0, x_max, xi_name))

for varN, xi_name in enumerate(Ow):                          # declaring load doispatch variables
    variables.append(solver.NumVar(0.0, x_max, xi_name))

for varN, xi_name in enumerate(Aijk):                          # declaring load doispatch variables
    variables.append(solver.NumVar(0.0, x_max, xi_name))
    
print('Number of variables created =', solver.NumVariables())
    #for variable in variables:
        #print('%s = %d' % (variable.name(), variable.solution_value())

Number of variables created = 225


## Declare constraints

In [225]:
#DECLARE CONSTRAINTS
#b = np.zeros((1, len(A)))

for rowN, row in enumerate(A):
    left_side = None
    for colN, coeff in enumerate(row):
        if coeff == 0:
            continue
        if left_side is None:
            left_side = coeff*variables[colN]
        else:
            left_side += coeff*variables[colN]
    if left_side is None and b[rowN] < 0:
    #if left_side is None and b[0,rowN] < 0:
        raise ValueError('Constraint ' + str(rowN) + ' cannot be satisfied!')
    if left_side is not None:
        #solver.Add(left_side <= t[0,rowN])
        solver.Add(left_side <= b[rowN])
        
print('Number of constraints added =', solver.NumConstraints())

Number of constraints added = 439


## Declare Objective Function

In [226]:
# declare objective function: x0+x1+x2+x3+x4+......

#C1 = [variables[0], variables[1], variables[2], variables[3], variables[4], variables[5]]
#C = []
cost = None
coeffs = [1.0 for _ in X1]
#coeffs[3] = 100: coeffs[8] = 100: coeffs[1] = 100: coeffs[6] = 100
for i in range (0, len(X1)):
    cost = variables[i]*coeffs[i] if i == 0 else cost + variables[i]*coeffs[i]
    #C.append(variables[i])
    #print (variables[i])
#objective = sum(C)
solver.Minimize(cost)

# Weights for prioritizing different variables
#wgt_X1 = 1.0
#X1_coeffs = [ wgt_X1 for _ in X1 ]
#for coeffN, coeff in enumerate(X1_coeffs):
#    if coeff != 0:
#        C += coeff*X1[coeffN] 

#solver.Maximize(x0 + x1 + x2 + x3 + x4 + x5 + x6 +x7 + x8 + x9)   

## Invoke the solver

In [227]:
result_status = solver.Solve()
    # The problem has an optimal solution.
assert result_status == pywraplp.Solver.OPTIMAL
print('Objective value =', solver.Objective().Value())
for k in range (0,V):
    print("cycle", k)
    for i in range (0, edges):
        print(X[edges*k+i], variables[edges*k+i].solution_value())

Objective value = 10.0
cycle 0
x0 0.0
x1 0.0
x2 0.0
x3 0.0
x4 1.0
x5 1.0
x6 0.0
x7 1.0
x8 0.0
cycle 1
x9 0.0
x10 1.0
x11 1.0
x12 1.0
x13 0.0
x14 0.0
x15 0.0
x16 0.0
x17 1.0
cycle 2
x18 1.0
x19 0.0
x20 0.0
x21 0.0
x22 0.0
x23 0.0
x24 0.0
x25 1.0
x26 1.0


In [166]:
for k in range (0,V):
    print("cycle", k)
    for i in range (0, len(X)):
        print(X[edges*k+i], variables[edges*k+i].solution_value())

cycle 0
x0 0.0
x1 0.0
x2 0.0
x3 0.0
x4 0.0
x5 0.0
x6 0.0
x7 0.0
x8 0.0
x9 1.0
x10 0.0
x11 0.0
x12 0.0
x13 1.0
x14 1.0
x15 0.0
x16 2.0
x17 1.0
x18 0.0
x19 1.0
x20 1.0
x21 1.0
x22 0.0
x23 0.0
x24 0.0
x25 0.0
x26 1.0
k0 0.0
k1 0.0
k2 0.0
k3 0.0
k4 0.0
k5 0.0
k6 2.0
k7 1.0
k8 0.0
k9 0.0
k10 2.0
k11 1.0
k12 0.0
k13 1.0
k14 1.0
k15 1.0
k16 1.0
k17 0.0
Ow0 0.0
Ow1 2.9999999999999503
Ow2 0.0
Ow3 0.0
Ow4 0.0
Ow5 28.000000000000057
Ow6 11.0
Ow7 0.0
Ow8 0.0
Ow9 0.0
Ow10 51.00000000000006
Ow11 37.99999999999994
Ow12 0.0
Ow13 19.00000000000005
Ow14 33.0
Ow15 44.00000000000001
Ow16 3.999999999999943
Ow17 0.0
A000 0.0
A001 11.0
A002 0.0
A010 0.0
A011 0.0
A012 0.0
A020 0.0
A021 0.0
A022 0.0
A030 0.0
A031 0.0
A032 0.0
A040 0.0
A041 0.0
A042 0.0
A050 0.0
A051 11.0
A052 0.0
A060 0.0
A061 0.0
A062 0.0
A070 0.0
A071 11.0
A072 0.0
A080 0.0
A081 0.0
A082 0.0
A100 0.0
A101 0.0
A102 0.0
A110 0.0
A111 0.0
A112 19.00000000000005
A120 0.0
A121 0.0
A122 0.0
A130 0.0
A131 0.0
A132 0.0
A140 0.0
A141 0.0
A142 0.0
A15

IndexError: list index out of range

In [ ]:
assert solver.VerifySolution(1e-7, True)

print('Solution:')
print('Objective value =', solver.Objective().Value())
print('Number of variables =', solver.NumVariables())
print('Number of constraints =', solver.NumConstraints())
print('\nAdvanced usage:')
print('Problem solved in %f milliseconds' % solver.wall_time())
print('Problem solved in %d iterations' % solver.iterations())
print('Problem solved in %d branch-and-bound nodes' % solver.nodes())